In [83]:
import json
import os
from os import path
import glob
from collections import defaultdict

In [84]:
log_dir = "/home/shtoshni/Research/events/kenton_coref/e2e-coref/logs/best"
output_file = path.join(log_dir, "eval_output.jsonlines")

output_dir = path.join(log_dir, "html")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [85]:
HTML_START = '<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8"></head><body>'

corr_tag = '<div style="border:2px; display:inline; border-style: solid; border-color: blue; padding: 10px; padding-right: 4px; padding-left: 4px">'
pred_tag = '<div style="border:2px; display : inline; border-style: solid; border-color: red; padding:10px; padding-right: 4px; padding-left: 4px">'

end_tag = '</div>'

In [86]:
doc_list = []

with open(output_file) as f:
    for line in f.readlines():
        # Load the output
        output_dict = json.loads(line.strip())
        
        doc_key = output_dict["doc_key"]
#         print(doc_key)
        doc_key = doc_key.split("/")[1]
        doc_list.append(doc_key)
        print(doc_key)
        
        sentences = output_dict["sentences"]
        source_doc = []
        
#         sent_indices = set()
#         offset = 0
        for sent in sentences:
            sent[-1] = sent[-1] + "\n"
            source_doc.extend(sent)
#             sent_indices.add(offset + len(sent))
#             offset += len(sent)
        
        clusters = output_dict["clusters"]
        pred_clusters = output_dict["predicted_clusters"]
        
        entity_list = []
        html_tag_list = defaultdict(list)

        # Get all the entity info
        for cluster_set, cluster_type in zip([clusters, pred_clusters], ["gt", "pred"]):
            # Get the html tag based on type of cluster
            if cluster_type == 'gt':
                html_tag = corr_tag
            elif cluster_type == 'pred':
                html_tag = pred_tag
                
            for cluster_id, cluster in enumerate(cluster_set):
                for (span_start, span_end) in cluster:
                    entity_list.append((cluster_type, cluster_id, (span_start, span_end)))


                    html_tag_list[span_start].append((html_tag, 'start' + '-' + cluster_type, str(cluster_id) + '_' + cluster_type))
                    html_tag_list[span_end].append((end_tag, 'end' + '-' + cluster_type, str(cluster_id) + '_' + cluster_type))

        for key in html_tag_list:
            # Sort all the HTML tags so that the end tags occur before the start tags
            html_tag_list[key] = sorted(html_tag_list[key], key=lambda x: x[1])


        html_string = HTML_START + '<div style="line-height: 3">'

        for idx, token in enumerate(source_doc):
            token = token.replace("<", "~")
            token = token.replace(">", "^")
            
            if idx in html_tag_list:
                tags = html_tag_list[idx]
                for tag, tag_type, cluster_id in tags:
                    if 'end' in tag_type:
                        html_string += "<sub>" + str(cluster_id) + "</sub>" 
                    html_string += tag
            
#             if idx in sent_indices:
#                 html_string += '\n'
            html_string += token + " "


        html_string += "</div></body></html>"
        html_string = html_string.replace("\n", "\n<br/>")
        html_string = html_string.replace("~", "&lt;")
        html_string = html_string.replace("^", "&gt;")
        with open(path.join(output_dir, doc_key + ".html"), "w") as f:
            f.write(html_string)

NYT_ENG_20131029.0091
NYT_ENG_20130613.0153
17a2dc40635ec239e9e16d10b6dd45e8.mpdf
NYT_ENG_20130424.0047
96bf72399b104346f3e79022e0c08e5a.mpdf
57026b7bcb8f855de3e26d572db35285
soc.culture.china_20050203.0639
PROXY_AFP_ENG_20020406_0538
PROXY_AFP_ENG_20020128_0449


In [87]:
index_html = HTML_START + '<ol type="1">'

for doc_key in doc_list:
    html_output_file = doc_key + ".html"
    
    index_html += '<li> <a href="{}", target="_blank">'.format(html_output_file) + doc_key + '</a></li>\n'
    

index_html += '</ol>\n</body>\n</html>'
with open(path.join(output_dir, "index.html"), "w") as g:
    g.write(index_html)